![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XlmRoBertaSentenceEmbeddings.ipynb)

## Import XlmRoBertaSentenceEmbeddings models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for XlmRoBertaSentenceEmbeddings from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use XlmRoBertaSentenceEmbeddings models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLMRobertaTokenizer requires the `SentencePiece` library, so we install that as well

In [2]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 55.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which i

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [xlm-roberta-base](https://huggingface.co/xlm-roberta-base) model from HuggingFace as an example
- In addition to `TFXLMRobertaModel` we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- Since `xlm-roberta-base` model is PyTorch we will use `from_pt=True` param to convert it to TensorFlow

In [3]:
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel
import tensorflow as tf

# xlm-roberta-base
MODEL_NAME = 'xlm-roberta-base'

XLMRobertaTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFXLMRobertaModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFXLMRobertaModel.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

try downloading TF weights


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {MODEL_NAME}

total 1086384
-rw-r--r-- 1 root root        673 Mar  1 22:30 config.json
drwxr-xr-x 3 root root       4096 Mar  1 22:30 saved_model
-rw-r--r-- 1 root root 1112441536 Mar  1 22:31 tf_model.h5


In [5]:
!ls -l {MODEL_NAME}/saved_model/1

total 9076
drwxr-xr-x 2 root root    4096 Mar  1 22:30 assets
-rw-r--r-- 1 root root      54 Mar  1 22:30 fingerprint.pb
-rw-r--r-- 1 root root  165621 Mar  1 22:30 keras_metadata.pb
-rw-r--r-- 1 root root 9111290 Mar  1 22:30 saved_model.pb
drwxr-xr-x 2 root root    4096 Mar  1 22:30 variables


In [6]:
!ls -l {MODEL_NAME}_tokenizer

total 4960
-rw-r--r-- 1 root root 5069051 Mar  1 22:29 sentencepiece.bpe.model
-rw-r--r-- 1 root root     279 Mar  1 22:29 special_tokens_map.json
-rw-r--r-- 1 root root     502 Mar  1 22:29 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `sentencepiece.bpe.model` file from the tokenizer
- all we need is to copy `sentencepiece.bpe.model` file into `saved_model/1/assets` which Spark NLP will look for

In [7]:
# let's copy sentencepiece.bpe.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {MODEL_NAME}/saved_model/1/assets

## Import and Save XlmRoBertaSentenceEmbeddings in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [8]:
#please Restart here to clear up RAM
while True:
  pass


KeyboardInterrupt: 

In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.0
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [2]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlmRoBertaSentenceEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `XlmRoBertaSentenceEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [4]:
from sparknlp.annotator import *

MODEL_NAME = 'xlm-roberta-base'

# All these params should be identical to the original ONNX model
xlm_roberta = XlmRoBertaSentenceEmbeddings.loadSavedModel(f"{MODEL_NAME}/saved_model/1", spark)\
    .setInputCols(["sentence"])\
    .setOutputCol("xlm_roberta")\
    .setCaseSensitive(True)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [5]:
xlm_roberta.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [6]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX XlmRoBertaSentenceEmbeddings model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [7]:
! ls -l {MODEL_NAME}_spark_nlp

total 1099988
drwxr-xr-x 3 root root       4096 Mar  1 22:39 fields
drwxr-xr-x 2 root root       4096 Mar  1 22:39 metadata
-rw-r--r-- 1 root root    5069051 Mar  1 22:39 xlmroberta_spp
-rw-r--r-- 1 root root 1121302747 Mar  1 22:39 xlmroberta_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaSentenceEmbeddings model 😊

In [8]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

xlm_roberta_loaded = XlmRoBertaSentenceEmbeddings.load(f"{MODEL_NAME}_spark_nlp")\
    .setInputCols(["sentence"])\
    .setOutputCol("xlm_roberta")

pipeline = Pipeline(
    stages = [
        document_assembler,
        sentencerDL,
        xlm_roberta_loaded
  ])

data = spark.createDataFrame([['William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor,and philanthropist.']]).toDF("text")
model = pipeline.fit(data)
result = model.transform(data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]


In [9]:
result.selectExpr("explode(xlm_roberta.embeddings[0]) as embeddings").show()

+------------+
|  embeddings|
+------------+
| -0.05523606|
|  0.21861903|
| 0.079868846|
|   0.5373767|
|  0.08400798|
|  0.38843948|
|  0.38681212|
| -0.36239216|
|  0.21800546|
|  -0.1326824|
|-0.039364785|
|  0.13006476|
|  0.31846768|
|   0.3994937|
| -0.40145183|
| -0.20561102|
|  0.35796887|
|  0.33135167|
| 0.014850351|
| -0.21051204|
+------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of XlmRoBertaSentenceEmbeddings models from HuggingFace 🤗 in Spark NLP 🚀
